In [ ]:
# Title 추가
# train['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.')
# display(train)

# 필요없는 column제거
train.drop(['PassengerId','Name','Ticket','Fare','Cabin'], axis=1, inplace=True)

# 성별처리
sex_mapping = { 'male' : 0, 'female' : 1 }
train['Sex'] = train['Sex'].map(sex_mapping)

# n_of_family
train['FamilyNum'] = train['SibSp'] + train['Parch']

# 필요없는 column제거
train.drop(['SibSp','Parch'], axis=1, inplace=True)

# 결측치 확인
train.isnull().sum()

# Embarked 결측치 처리
train['Embarked'] = train['Embarked'].fillna('S')
# Embarked 값 숫자 처리
embarked_mapping = { 'S':0, 'C':1, 'Q':2}
train['Embarked'] = train['Embarked'].map(embarked_mapping)

# Age 결측치 처리
train['Age'] = train['Age'].fillna(train['Age'].mean())


# Age에 대한 Binning 처리(numeric value -> categorical value)
train.loc[train['Age'] < 8, 'Age'] = 0
train.loc[(train['Age'] >= 8) & (train['Age'] < 20), 'Age'] = 1
train.loc[(train['Age'] >= 20) & (train['Age'] < 65), 'Age'] = 2
train.loc[train['Age'] >= 65, 'Age'] = 3

display(train)
train.isnull().sum()


In [26]:
# Multinomial Classification
# BMI 지수에 대한 데이터를 가지고 학습 후 예측까지 진행해 볼꺼예요!
# BMI 지수는 키와 몸무게를 가지고 저체중, 정상, 과체중, 비만을 판단하는 지수.
# BMI = 자신의 몸무게(kg) / 키의 제곱(m) 
#       18.5 이하 => 저체중
#       18.5 ~ 23 => 정상
#       23 ~ 25 => 과체중
#       25 ~ => 비만
#       키가 180cm, 몸무게 80kg인 경우
#       80 /1.8^2(3.24) = 24.69 => 과체중!~
# 우리가 하려는 건 식이 아니라... BMI지수를 조사한 데이터가 있어요!
# 이걸 학습해서 예측을 통해 나의  BMI 지수를 알아보려고 합니다.!!
# 단, 제공하는 데이터는 4가지 분류가 아니라 3가지 분류로 나누어져 있어요!
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# Raw Data Loading
df = pd.read_csv('./data/bmi.csv', skiprows=3)
# display(df)

# 결측치부터 확인!
# df.isnull().sum()   # 결측치는 없어요!!

# 이상치 확인
# zscore = 1.8
# height의 이상치를 확인하고 싶어요!
# df.loc[np.abs(stats.zscore(df['height'])) >= zscore,:] # height의 이상치는 없어요!
# df.loc[np.abs(stats.zscore(df['weight'])) >= zscore,:] # weight의 이상치는 없어요!
# df.loc[np.abs(stats.zscore(df['label'])) >= zscore,:] # label의 이상치는 없어요!

# Data Split
# Train, Test 두 부분으로 분할. 분리하는 비율은 7:3으로 분리.
# 나중에 Train부분은 k-fold cross validation을 진행할 꺼예요!
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df[['height','weight']],df['label'],test_size=0.3, random_state=0)

# Normalization
scaler = MinMaxScaler()  # scaler객체를 생성해요!
scaler.fit(x_data_train) # scaler객체에 최대, 최소와 같은 정보가 들어가요!

x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

del x_data_train   # 혼동을 방지하기 위해서 변수를 삭제
del x_data_test

# sklearn 구현은 매우매우 간단!!
# Model을 생성하고 학습시키고 예측할꺼예요
model = LogisticRegression()
model.fit(x_data_train_norm,t_data_train)

# 우리 model의 정확도를 측정해야 해요!
# cross validation
kfold = 10
kfold_score = cross_val_score(model,x_data_train_norm,t_data_train,cv=kfold)
print('### cross validation ###')
print('### cross validation score###')
print('score : {}'.format(kfold_score))
print('평균: {}'.format(kfold_score.mean()))

# 최종모델평가
predict_val = model.predict(x_data_test_norm)  # 테스트데이터로 예측값을 구해요!
acc = accuracy_score(predict_val,t_data_test)
print('우리  Model의 최종 Accuracy : {}'.format(acc))

# Predict

height = 188
weight = 100
my_state = [[height,weight]]
my_state_val = model.predict(scaler.transform(my_state))

print(my_state_val)  # [1]

### cross validation ###
### cross validation score###
score : [0.98       0.98642857 0.985      0.97642857 0.98642857 0.98428571
 0.98714286 0.97714286 0.97714286 0.98642857]
평균: 0.9826428571428572
우리  Model의 최종 Accuracy : 0.9845
[2]
